# Seat Geek to Hubspot ETL

This code describes the process to transfer ticket transcation data from Seat Geek export to Hubspot. This data supports ROI tracking on marketing assets and customer segmentation for sales/marketing strategies.


### Step 1 - Extract data from Unify

Depending on the upload, if this is to be done after each game you will head to Unify, go to reports, select Seats Sold (1 per row) and download the file. For the upload I am doing currently we are backfilling all of 2024 so we will be using all events in the report. But otherwise, select the date range that your event took place and then click the events drop down and select your event. Download this file as a csv.

This file contains every seat for that game including seat, customer, and price information.

We will use this data in order to build out our individual game transactions in HubSpot as well as import the customers that were created

This data will come in the raw export form, no adjustments are needed before running this code, simply update the csv you are reading and run. All columns will be cleaned to fit the correct names and those that are unnecessary will be dropped

At the end of this step you will be left with two dataframes, one for tickets and one for customers. The one for tickets will be used to create deals and line items to be imported into HubSpot. The customers will be cleaned and assigned the properties needed them imported into HubSpot as well. <b> Customers must be imported into HubSpot before Deals </b> 

In [ ]:
import pandas as pd

tickets = pd.read_csv('Seats Sold (1 Seat Per Row) 2024-11-13 11_45_40.csv', low_memory=False)

Season = 2024

# Update column names step by step
tickets.columns = tickets.columns.str.replace("MainTable_DetailsValue", "", regex=False)
tickets.columns = tickets.columns.str.replace("TikOwner", "", regex=False)
tickets.columns = tickets.columns.str.replace("1", "", regex=False)
tickets.columns = tickets.columns.str.replace("AddressLine2_8", "AddressLineX", regex=False)
tickets.columns = tickets.columns.str.replace("[023456789_]", "", regex=True)
tickets.columns = tickets.columns.str.replace("AddressLineX", "AddressLine2", regex=False)
tickets.columns = tickets.columns.str.replace("Descr", "", regex=False)

# Columns to keep
columns_to_keep = [
    "EventDate", "EventName", "FirstName", "LastName", "AddressLine", "AddressLine2", 
    "City", "State", "ZipCode", "Country", "Email", "HomePhone", 
    "BusinessPhone", "MobilePhone", "Area", "Sector", "Row", "Seat", "SeatType", 
    "PriceLevel", "PriceTypeGroup", "PriceType", "TransactNum", "TransactDate", 
    "TotalSaleValue", "IsScanned"
]

# Perform column selection
tickets = tickets[columns_to_keep]

# Creating the 'customers' dataframe with the specified columns
customers_columns = [
    "FirstName", "LastName", "AddressLine", "AddressLine2", "City", "State", 
    "ZipCode", "Country", "Email", "HomePhone", "BusinessPhone", "MobilePhone", "PriceTypeGroup", "PriceType"
]

# Create the 'customers' dataframe
customers = tickets[customers_columns]

# Drop the specified columns from tickets except 'Email'
columns_to_drop = [
    "FirstName", "LastName", "AddressLine", "AddressLine2", "City", "State", "SeatType",
    "ZipCode", "Country", "HomePhone", "BusinessPhone", "MobilePhone", "Area", "Row", "Seat"
]

# Update the 'tickets' dataframe
tickets = tickets.drop(columns=columns_to_drop)

### Step 2 - Import New Contacts to Hubspot

We must first migrate our unique contacts to HubSpot to ensure we don't have errors upon importing deals, there are two types of errors we must account for:

1. <b> Invalid Email </b> -- This we solve above when correcting the email domains, HubSpot will only import valid email addresses
2. <b> Invalid Duplicate ID </b> -- This error occurs when a contact is created in an import then appears again in the import. Since we are importing multiple deals associating to contacts we are at risk of this error. This only occurs when a contact is created in the import so by first importing the contacts then the deals we ensure that we won't create any contacts when importing deals removing this risk

When importing these contacts we want to ensure we are including all the data exported from Unify into HubSpot. We also want to ensure this data is clean and that contacts are being put in the correct business unit. The exported file from Step 1 has the correct columns needed. This code below will clean and match them to get us an export of unique emails with cleaned information

1. <b> Find Business units </b> -- Add the LUFC business unit - this is necessary for import into HubSpot
2. <b> Add Contact owner </b> -- Add the contact owners
3. <b> Clean customer emails </b> -- Clean emails, ensure no empty ones
4. <b> Clean customer columns </b> -- We then want to clean the customers data that we will be importing into HubSpot. This includes formating name, address, and phone number data as well as removing the columns no longer needed
5. <b> THERE WILL BE MORE STEPS IN THE FUTURE </b> -- For this offseason import it is not import but when moving towards imports after each game when the season is going this code will need to be adjusted to have parts after. What these entail is you need to import all buyers which is what this will create, but then you also want to import the single game buyers and break them down by phone or no phone. To do this you would import the customers file that's outputted. Then export all contacts from that import including their "Phone Number" column and the "Contact Owner - LUFC". These would then get matched back to the original import based on Email. You would merged the Phone Number column and then filter on PriceTypeGroup is Single. This would then be split into those with phone numbers and those without and each of these would be imported and if a contact had contact ownership of the info account, that would be reassigned to one of their account executives. Each of these new imports, one with phone numbers one without, would be imported <b> with overwriting on the "Contact Owner - LUFC" </b>

After these steps are followed you are left with a file to import into HubSpot which will ensure all of our customers contacts are in the system before importing the deals which will allow them to match. When importing, match each column to its corresponding HubSpot column, all of these are self explanatory when matching. It is important that you <b> do not overwrite </b> is selected for all columns

##### <b> Do not overwrite columns on contact import </b>

In [ ]:
import re

# Drop duplicate emails
customers = customers.drop_duplicates(subset='Email')

# Function to filter out malformed emails (doesn't contain @ symbol)
def format_emails(email):
    if pd.notna(email) and '@' in email:
        return email
    return ''

# Function to fix invalid emails
def format_invalid_emails(email):
    com_variations = [',com', '.cim', '.ocm', '.C0M', '.comm', '. COM', '.coom', '.cop', '.oom', '.lcom', '.con', '.co,m', '..com', '.cpm', '.cpom', '.col', '.co,', '.xom', '.fom', '.coke', 'cok', '.come', '..com', 'clom', '.com.']
    net_variations = [',net', '.met', '.ent', '.nat', '.nt', '.ner', '.ney', '.lnet', '. net', '.nety', '.neto', '.ncet', '/net', '.nedt', '.nett', '.nbet']
    
    # Skip processing if the email is NaN or empty
    if pd.isna(email) or email == '':
        return email
    
    # Remove spaces from the email
    email = email.replace(" ", "")
    
    # Fix .com variations
    for variation in com_variations:
        if variation in email:
            email = email.replace(variation, ".com")
            break
    
    # Fix .net variations
    for variation in net_variations:
        if variation in email:
            email = email.replace(variation, ".net")
            break

    # Remove consecutive dots
    while '..' in email:
        email = email.replace('..', '.')
    
    return email


# Function to format phone numbers
def format_phone_number(phone):
    if pd.isna(phone) or isinstance(phone, str) and not re.search(r'\d', phone):
        return None
    else:
        clean_phone = re.sub(r'[^\d]', '', str(phone))  # Remove all non-digit characters
        if re.match(r'(\d{10})$', clean_phone):
            # category 1, complete number
            formatted_number = f'+1{clean_phone}'
            return formatted_number
        else:
            return None

# Function to format zip codes
def format_zip_codes(zip_code):
    if pd.isna(zip_code) or not isinstance(zip_code, str):
        return ''
    zip_code = re.sub(r'\D', '', zip_code)  # Remove non-digit characters
    if len(zip_code) >= 5:
        return zip_code[:5]  # Return the first 5 characters
    return ''

# Create the 'Business Unit' column
customers['Business Unit'] = ';Loudoun United FC'
customers['Contact Owner - LUFC'] = 'info@loudoununitedfc.com'

# Apply the functions to clean 'Email', skipping NaN values
customers['Email'] = customers['Email'].apply(lambda x: format_emails(x) if pd.notna(x) else x)
customers['Email'] = customers['Email'].apply(lambda x: format_invalid_emails(x) if pd.notna(x) else x)

# Drop rows with no email
customers = customers[customers['Email'].notna() & (customers['Email'] != '')]

# Set country to US (comes as USA)
customers['Country'] = 'US'

# Ensure columns are in title case
customers['FirstName'] = customers['FirstName'].str.title()
customers['LastName'] = customers['LastName'].str.title()
customers['AddressLine'] = customers['AddressLine'].str.title()
customers['AddressLine2'] = customers['AddressLine2'].str.title()
customers['City'] = customers['City'].str.title()

# Apply the function to the 'Zip' column to clean
customers['ZipCode'] = customers['ZipCode'].apply(format_zip_codes)

# Apply the function to the 'Phone' column to clean
customers['HomePhone'] = customers['HomePhone'].apply(format_phone_number)
customers['BusinessPhone'] = customers['BusinessPhone'].apply(format_phone_number)
customers['MobilePhone'] = customers['MobilePhone'].apply(format_phone_number)

# Creating the Phone Number and filling it with HomePhone, or MobilePhone, or BusinessPhone
customers['Phone Number'] = customers['HomePhone'].fillna(customers['MobilePhone']).fillna(customers['BusinessPhone'])

# Drop the 'HomePhone' column
customers = customers.drop(columns=['HomePhone'])

# Drop duplicate emails
customers = customers.drop_duplicates(subset='Email')

# Save file for Import
customers.to_csv('Import Customers.csv')

### Step 3 - Deal Data Transformation

We perform several data transformations to improve data quality and make the data acceptable for import to Hubspot

1. <b> Email domain corrections </b> -- Replace invalid domain exentions like ".comm" with ".com" or their likely equaivalent
2. <b> Convert tickets scanned to 1,0 </b> -- Reconfigure [IsScanned] to binary to sum when grouping to get the total number of tickets scanned per transaction
3. <b> Set ticket price to numeric </b> -- Originally had issues with some prices being in different formats, this ensures there are no issues when calculating total price
4. <b> Set price for certain ticket types to 0 </b> -- Certain deals are already tracked in HubSpot, we set these types to zero as to not double count our revenues
5. <b> Adjusting close date </b> -- Some transactions have tickets without close dates, these are populated with the same date as the rest of the transaction or the event date as when importing to HubSpot if there is no closed date on a closed won deal it will default to the create date and we don't want deals to have close dates after the events for individual tickets
6. <b> Remove tickets without emails </b> -- Our purpose for this import is to calculate ROI and support sales. There are few deals without emails so it is not impactful on revenue reporting. This is important to remove as otherwise it will cause issues in our imports


In [ ]:
# Extract the part of EventName before " at " and then rename to Opponent
tickets['EventName'] = tickets['EventName'].str.split(" at ").str[0]

tickets = tickets.rename(columns={
    "EventName": "Opponent",
    "TransactDate": "Close Date",
    "Sector": "Section",
    "IsScanned": "IsScanned",
    "TotalSaleValue": "Amount",
    "PriceTypeGroup": "Individual Deal Type"
})

# Ensure 'Amount' is a numeric float
tickets['Amount'] = pd.to_numeric(tickets['Amount'], errors='coerce').astype(float)

# Adjust the Amount based on Individual Deal Type
tickets.loc[tickets['Individual Deal Type'] != 'Single', 'Amount'] = 0

# Set any remaining blank Amount values to 0
tickets['Amount'] = tickets['Amount'].fillna(0)

# Change 'Yes' to 1 and 'No' to 0 in the "IsScanned" column
tickets['IsScanned'] = tickets['IsScanned'].map({"Yes": 1, "No": 0})

# Create a new column 'line_item_name' as PriceLevel + PriceType
tickets['line_item_name'] = tickets['PriceLevel'] + " " + tickets['PriceType']

# Apply the functions, skipping NaN values
tickets['Email'] = tickets['Email'].apply(lambda x: format_emails(x) if pd.notna(x) else x)
tickets['Email'] = tickets['Email'].apply(lambda x: format_invalid_emails(x) if pd.notna(x) else x)

# Drop rows where Email is unknown (NaN or empty string)
tickets = tickets[tickets['Email'].notna() & (tickets['Email'] != '')]

tickets.to_csv('Dont Import Tickets.csv')

## Step 4 - Transform Tickets into Transcations for Import

Now we have all our tickets cleaned up we are ready to group these into their specific transactions which will be the deals we will import

1. <b> Define prioritization of tickets </b> -- We want to prioritize what ticket type will assume the primary name of the [TicketType] in the Deal. When we import our Deals into HubSpot we can only have one TicketType
2. <b> Group into transactions </b> -- Grouping is done by [TransactionID] and [Event Date] so we can see the specific breakdown of games attended. After grouping the required HubSpot properties are added and we set the [Deal Owner] and [BusinessUnit] to the correct name. These Deals are then named and ready for import into HubSpot.
3. <b> Add properties needed for import </b> -- Not all the properties we want to include or that are needed for an import into HubSpot are in the data so after we group into transactions we want to add these properties in

All column names match those they are importing to but one area to keep a note on is when importing the [Close Date] property will default to be a contact property. <b> Close Date must be switched to the deal property under the same name or the deal closed dates will be set to the import date. </b>

If you get an error saying Duplicate Association ID for the deals after import these deals were still imported just without an association and you will need to go into each deal and associate them manually. This is not an error that should typically come up in small imports.
    

In [ ]:
tickets = tickets.sort_values(by=['Email', 'TransactNum'], ascending=[True, True])

# Define a function to prioritize TicketType
def prioritize_ticket_type(ticket_types):
    # Define the priority order
    priority_order = ['Single', 'Group', 'Season', 'Season Comp', 'Comp']
    
    # Remove NaN values and sort based on the priority
    ticket_types = [t for t in ticket_types if pd.notna(t)]
    for priority in priority_order:
        if priority in ticket_types:
            return priority
    return 'Other'  # Default if none of the types match

deals = tickets.groupby(['TransactNum', 'EventDate', 'Email']).agg(
    Closed_Date=('Close Date', 'first'),
    Opponent=('Opponent', 'first'),
    Section=('Section', 'first'),
    line_item_name=('line_item_name', 'first'),
    Individual_Deal_Type=('Individual Deal Type', prioritize_ticket_type),
    Amount=('Amount', 'sum'),
    Number_of_Tickets=('Amount', 'size'),
    Number_of_Tickets_Scanned=('IsScanned', 'sum')
).reset_index()

# Format 'Amount' to two decimal places
deals['Amount'] = deals['Amount'].round(2)

# Add the new columns
deals['Pipeline'] = 'Individual Tickets Pipeline'   # Set Pipeline as 'Individual Tickets'
deals['Deal Stage'] = 'Closed Won'                  # Set Deal Stage as 'Closed Won'
deals['Deal type'] = 'Individual Ticket'            # Set Deal type as 'Individual Ticket
deals['Season'] = Season                            # Set Season as the current season
deals['Ticketing Platform'] = 'SeatGeek'            # Set the Ticketing Platform
deals['Deal Owner'] = 'info@loudoununitedfc.com'    # Set Deal Owner
deals['Business units'] = 'Loudoun United FC'       # Set the business unit

# Rename columns in the Deals dataframe
deals.rename(columns={
    'TransactNum': 'SeatGeek transaction number',
    'Individual_Deal_Type': 'Individual Ticket Type'
}, inplace=True)

# Name the deal
deals['Deal Name'] = deals['Season'].astype(str) + ' ' + deals['Opponent'].astype(str) + ' ' + deals['Number_of_Tickets'].astype(str) + ' ' + deals['Individual Ticket Type'] + ' Tickets'

deals.to_csv('Import Deals.csv')


# Step 5 - Export Deals from HubSpot and Merge with Line Items

When updating deals in HubSpot you need the Record ID of the deal. Since line items will be added onto the deals we need to <b> export the deals we created along with their Record ID, SeatGeek transaction number, Associated Contact and Event Date </b> to merge the Record IDs for the Deals to the line items. 

In doing this we are using the <b> tickets </b> dataframe that we had saved previously after cleaning our data <br>
Export the deals from HubSpot and save them as <b> export.csv </b> in the same location as the code

1. <b> Match the Event Date, Transaction Number, Email </b> -- Since we are merging on [Event Date] we need to ensure that the formats of the event dates are the same across dataframes
2. <b> Merge data </b> -- Here we merge the data to get the Record ID of deals associated with the line items. We do a left join because we know that for each deal there must be associating line items as deals all have at least one ticket associated with them and the line items are representative of the tickets. It is import to <b> check all line items have a matching deal </b> before importing
3. <b> Create Line Item properties </b> -- There are 3 required properties for creating a line item in HubSpot. These 3 properties are Name, Quantity, and Price. Line item name was previously created and is something we are grouping by. We follow this up with creating the Price and Quantity properties. These take all the tickets with the same name from the transaction and counts the number of tickets and takes the average price in order to have the amount in our line items match the amount of the deal. <b> GROUP BY </b>

Once these are merged and added we can export this file for import into HubSpot. HubSpot can't take line items with negative values so it is important to set line items with values less than zero to equal zero before importing (there should be none anyway)

To import into HubSpot:
1. Select Deals and Line Items
2. Set to update Deals and create Line Items
3. Select Record ID as a deal property matching to Record ID
4. Set Name, Quantity, and Price to be line item properties under the same names
5. This should result in no errors in the preview and can go ahead and import

In [ ]:
exported_deals = pd.read_csv('export.csv')

# Extract the email part from the 'Associated Contact' column and create a new 'Email' column
exported_deals['Email'] = exported_deals['Associated Contact'].str.extract(r'\((.*?)\)')

# Convert all Email values in both dataframes to lowercase
tickets['Email'] = tickets['Email'].str.lower()
exported_deals['Email'] = exported_deals['Email'].str.lower()

# Convert 'EventDate' in tickets to match the format of 'Event Date' in exported_deals
tickets['EventDate'] = pd.to_datetime(tickets['EventDate'], errors='coerce').dt.date.astype(str)

# Ensure 'Event Date' in exported_deals is also in string format (if needed)
exported_deals['Event Date'] = pd.to_datetime(exported_deals['Event Date'], errors='coerce').dt.date.astype(str)

# Ensure 'seatgeek transaction number' in exported_deals is an integer
exported_deals['seatgeek transaction number'] = pd.to_numeric(exported_deals['seatgeek transaction number'], errors='coerce').fillna(0).astype(int)

# Ensure 'TransactNum' in tickets is an integer
tickets['TransactNum'] = pd.to_numeric(tickets['TransactNum'], errors='coerce').fillna(0).astype(int)

# Merge the dataframes and bring over 'Record ID' from exported_deals into tickets
tickets = tickets.merge(
    exported_deals[['Record ID', 'Event Date', 'seatgeek transaction number', 'Email']],
    left_on=['EventDate', 'TransactNum', 'Email'],
    right_on=['Event Date', 'seatgeek transaction number', 'Email'],
    how='left'
)

tickets.to_csv('line items test.csv')

# Group tickets by TransactNum, EventDate, Email, and line_item_name
line_items = tickets.groupby(['TransactNum', 'EventDate', 'Email', 'line_item_name']).agg(
    Quantity=('Amount', 'size'),  # Count of rows
    Price=('Amount', 'mean'),  # Average of Amount
    RecordID = ('Record ID', 'first')
).reset_index()

# Keep only the required columns
line_items = line_items[['RecordID', 'line_item_name', 'Quantity', 'Price']]

# Save the resulting dataframe
line_items.to_csv('Import Line Items.csv', index=False)

